In [ ]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")

In [ ]:
# This is an LLMChain to write a synopsis given a title of a play.
template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.

Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
# This is an LLMChain to write a review of a play given a synopsis.
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(
    chains=[synopsis_chain, review_chain], verbose=True
)

In [ ]:
review = overall_chain.run("Tragedy at sunset on the beach")

In [ ]:
print(review)

below code is for my thesis above is experiment

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks import get_openai_callback
from langchain.chains import SequentialChain
import PyPDF2
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")

In [ ]:
RESPONSE_JSON = {
    "tag" : "yes or no",
    "reason": "reason of being yes or no"
}

# Define your template
TEMPLATE = """
Abstract: {Abstract}
Keywords: {Keywords}

You are an expert in artificial intelligence (AI) and its various applications, particularly in the domain of food waste management. Your task is to analyze the provided abstract and keywords of a research paper to determine whether the paper is related to the application of AI in food waste management.

Please read the abstract and keywords carefully, then decide if the paper fits this criterion. If the paper is related to the application of AI in food waste management, tag it as "yes". If it is not, tag it as "no".

Example with given abstracts and keywords:

Abstract: Food security is one of the priorities of every country in the World. However, different factors are making it difficult to meet global targets on food security. Some unprecedented shocks are encumbering food security at the global level. Various interventions have been applied toward food security and artificial intelligence is one of the modern methods that is being used in various stages of the food system. In this paper, the application of artificial intelligence in the whole food production ecosystem ranging from crop production, livestock production, harvesting/slaughtering, postharvest management, food processing, food distribution, food consumption and food waste management is assessed. The objective of this research is to assess the application of artificial intelligence systems in all the stages of food systems. A systematic review was conducted by analyzing 110 articles after the screening of 450 articles based on the inclusion and exclusion criteria. The results indicated that various artificial intelligence algorithms are being applied to all the stages of the food system from crop/livestock production up to food or agro-waste management. © 2023 The Authors

Keywords: Agriculture; Artificial intelligence; Food processing; Food systems

json_response: {
    "tag": "yes",
    "reason": "The paper discusses the application of AI in food waste management along with other stages of the food system."
}

Abstract: This study investigates the impact of climate change on agricultural yields and the subsequent effect on food security. The paper examines historical data trends and provides projections for future crop production under various climate scenarios.

Keywords: climate change, agricultural yields, food security, data analysis, projections

json_response: {
    "tag": "no",
    "reason": "The paper discusses climate change and agricultural yields without mentioning the application of AI in food waste management."
}

Now, please analyze the following abstract and keywords and provide your response in the following JSON format.

### RESPONSE_JSON
{response_json}
"""

In [ ]:
# Create the prompt template
analysis_prompt = PromptTemplate(
    input_variables=["Abstract", "Keywords", "response_json"],
    template=TEMPLATE
)



In [ ]:
quiz_chain=LLMChain(llm=llm, prompt=analysis_prompt, output_key="analysis", verbose=True)

In [ ]:
TEMPLATE2="""
You are an expert of correcting JSON templates. You are here to check if analysis has following JSON Template.
{RESPONSE_JSON}
If Not then correct the formate and then give Analysis:
{analysis}
"""

In [ ]:
analysis_evaluation_prompt=PromptTemplate(input_variables=["RESPONSE_JSON", "analysis"], template=TEMPLATE)

In [ ]:
review_chain=LLMChain(llm=llm, prompt=analysis_evaluation_prompt, output_key="review", verbose=True)

In [ ]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["tag","RESPONSE_JSON","response_json", "Keywords", "Abstract"],
                                        output_variables=["analysis", "review"], verbose=True,)

In [ ]:
# Example data
papers = [
    {
        "Abstract": "The integration of machine learning algorithms in optimizing food supply chains has shown significant promise in reducing food waste. This paper explores various AI techniques used to predict spoilage rates and optimize inventory management in supermarkets, thereby minimizing waste and improving efficiency.",
        "Keywords": "artificial intelligence, machine learning, food waste management, supply chain optimization, spoilage prediction"
    },
    {
        "Abstract": "This study investigates the impact of climate change on agricultural yields and the subsequent effect on food security. The paper examines historical data trends and provides projections for future crop production under various climate scenarios.",
        "Keywords": "climate change, agricultural yields, food security, data analysis, projections"
    },
    {
        "Abstract": "Artificial intelligence is transforming the food industry by enabling better waste management strategies. This research highlights the use of AI in developing smart bins that automatically sort food waste and provide data-driven insights to reduce overall waste generation.",
        "Keywords": "artificial intelligence, smart bins, food waste management, data-driven insights, waste reduction"
    },
    {
        "Abstract": "Advancements in natural language processing (NLP) have significantly improved customer service automation in the retail sector. This paper reviews various NLP techniques and their applications in enhancing customer interactions and service efficiency.",
        "Keywords": "natural language processing, customer service, retail sector, automation, service efficiency"
    },
    {
        "Abstract": "This paper presents a comprehensive review of AI-based predictive analytics for food waste prevention in the hospitality industry. By analyzing consumption patterns and forecasting demand, AI can help reduce food waste in restaurants and hotels.",
        "Keywords": "artificial intelligence, predictive analytics, food waste prevention, hospitality industry, demand forecasting"
    },
    {
        "Abstract": "The study explores the role of blockchain technology in ensuring food traceability and safety. It discusses the implementation of blockchain systems in the food supply chain to enhance transparency and trust among consumers.",
        "Keywords": "blockchain technology, food traceability, food safety, supply chain, transparency"
    }
]

# Initialize the LLMChain (Assuming you have set up the LLMChain properly)
llm_chain = LLMChain(prompt=analysis_prompt, llm=llm)

# Process each paper
results = []
for paper in papers:
    response = llm_chain.run({
        "Abstract": paper["Abstract"],
        "Keywords": paper["Keywords"],
        # "json_response": json_response,
    })
    results.append(response)

# Print the results
for i, result in enumerate(results):
    print(f"Paper {i+1} Tag: {result}")

# Replace 'your_llm' with the instance of the language model you are using.


In [1]:
import pandas as pd
from openpyxl import load_workbook, Workbook
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import Ollama

# Initialize the language model
llm = Ollama(model="llama3")

# Define the prompt template
synopsis_template = """You are an AI expert. Given the title, abstract, and keywords of a paper, determine if the paper is related to the application of AI in food waste management. If it is, tag it as "yes". If not, tag it as "no". Provide the reason for your decision.

Title: {title}
Abstract: {abstract}
Keywords: {keywords}
Tag the paper: "yes" or "no"
Reason:"""
synopsis_prompt_template = PromptTemplate(input_variables=["title", "abstract", "keywords"], template=synopsis_template)

# Create the LLMChain for tagging and reasoning
synopsis_chain = LLMChain(llm=llm, prompt=synopsis_prompt_template)

# Load the data from the Excel file
input_file = "WebOfSceinece.xlsx"
output_file = "results.xlsx"

df = pd.read_excel(input_file)

# Create the output file with headers if it doesn't exist
try:
    book = load_workbook(output_file)
    sheet = book.active
except FileNotFoundError:
    book = Workbook()
    sheet = book.active
    # Write headers
    headers = df.columns.tolist() + ["Tag", "Reason"]
    # print(headers)
    print(headers)
    sheet.append(headers)
    book.save(output_file)


['Title', 'Year', 'DOI', 'Abstract', 'Keywords', 'Tag', 'Reason']


c:\Users\CEO\anaconda3\envs\pytorch_env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [2]:
# Process each row and append the result to the output file
for index, row in df.iterrows():
    title = row["Title"]
    abstract = row["Abstract"]
    keywords = row["Keywords"]
    
    # Generate the tag and reason using the LLM chain
    result = synopsis_chain.run({"title": title, "abstract": abstract, "keywords": keywords})
    print(result)
    
    # Parse the result
    lines = result.strip().split("\n")
    tag_lines = [line for line in lines if line.lower().startswith("tag:")]
    reason_lines = [line for line in lines if line.lower().startswith("reason:")]
    
    tag = tag_lines[0].split(":")[-1].strip().strip('"') if tag_lines else "No tag found"
    reason = reason_lines[0].split(":")[-1].strip().strip('"') if reason_lines else "No reason found"
        
    # Append the result to the output file
    row_data = list(row) + [tag, reason]
    sheet.append(row_data)
    book.save(output_file)

print("Processing complete. Results saved to", output_file)

# # Process each row and append the result
# for index, row in df.iterrows():
#     title = row["Title"]
#     abstract = row["Abstract"]
#     keywords = row["Keywords"]
    
#     # Generate the tag and reason using the LLM chain
#     result = synopsis_chain.run({"title": title, "abstract": abstract, "keywords": keywords})
#     print(result)
#     # Parse the result
#     lines = result.strip().split("\n")
#     tag = lines[-2].split(":")[-1].strip()
#     reason = lines[-1].split(":")[-1].strip()
    
#     # Update the result DataFrame
#     result_df.at[index, "Tag"] = tag
#     result_df.at[index, "Reason"] = reason

# # Write the result to the output file
# result_df.to_excel(output_file, index=False)

# print("Processing complete. Results saved to", output_file)


c:\Users\CEO\anaconda3\envs\pytorch_env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Tag: "yes"

Reason: The abstract clearly mentions that artificial intelligence (AI) is being applied in various stages of the food system, including food waste management. Additionally, the keywords are not specified, but the title and abstract suggest that AI applications in the agrifood sectors will be discussed, which includes food waste management.
Tag: "yes"

Reason: The paper explicitly mentions the application of artificial intelligence (AI) in machine vision for postharvest detection of disorders in horticultural products, which is directly related to food waste management. The authors discuss the use of AI for extracting information on postharvest disorders from complex image data obtained by modern imaging systems, highlighting its potential for improving market value and consumer appreciation. Additionally, the paper emphasizes the importance of reliable detection methods to avoid food loss, which is a key aspect of food waste management.
Tag: "yes"

Reason: Although the pap